In [12]:
import kinase_library as kl
testsequence='PSVEPPLsQETFSDL'

# Create a Substrate object with a target sequence (example: p53 S33)
s = kl.Substrate(testsequence)  # Lowercase 's' indicates a phosphoserine

# Predict potential kinase interactions for the substrate
s.predict()

c:\Users\ricea\AppData\Local\Programs\Python\Python312\Lib\site-packages\kinase_library\logomaker\__init__.py:45: SyntaxWarning: invalid escape sequence '\.'
  if re.match('demo_.*\.py', temp_name)]


KeyboardInterrupt: 

In [ ]:
"""Task 1 - to write code mutating first amino acid to every other possible amino acid"""
#how to replace one letter for another in test sequence.

#how to replace one letter for each amino acid



In [ ]:
mutated_sequences = [] #creates a list to store the mutated amino acids
amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # defines the standard 20 amino acids
for i, original_aa in enumerate(testsequence): #indexes the original amino acids 
    for new_aa in amino_acids: #creates a loop of all amino acids
        if new_aa != original_aa:  # Avoid replacing with itself
            if original_aa.isupper():  # only count mutations for uppercase amino acids
                mutated_seq = testsequence[:i] + new_aa + testsequence[i+1:] #creates a new sequence (mutated seq)
                mutated_sequences.append(mutated_seq) #stores mutated sequence

print(f'Mutated sequences (# sequences = {len(mutated_sequences)})')
print(mutated_sequences)
 #shows mutations at each position
for i, original_aa in enumerate(testsequence):
    if original_aa.isupper():  
        mutation_count = sum(1 for new_aa in amino_acids if new_aa != original_aa)
        print(f"Position {i} ({original_aa}): {mutation_count} mutations")
    else:
        print(f"Position {i} ({original_aa}): Skipped (Lowercase)")

        

In [ ]:
'''
Task2 - Run these sequences through the predict function and store the output as a csv file. Look at pandas.to_csv() for this.
Output should be 280 separate csv files. Don't worry about a naming convention yet. 
'''



In [33]:
import kinase_library as kl
import pandas as pd

testsequence = 'PSVEPPLsQETFSDL'  
amino_acids = "ACDEFGHIKLMNPQRSTVWY"
mutated_sequences = []  

for i, original_aa in enumerate(testsequence):  
    for new_aa in amino_acids:
        if new_aa != original_aa:  
            if original_aa.isupper(): #when i remove this, i get 286 mutations, when i leave this in i get 266.
                mutated_seq = testsequence[:i] + new_aa + testsequence[i+1:] 
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))  


print(f"Total mutations generated: {len(mutated_sequences)}") 

for i, original_aa, new_aa, mutated_seq in mutated_sequences: #below is gpt code, wanted to see what exceptions it threw up using try function
    try:
        s = kl.Substrate(mutated_seq)
        predictions = s.predict()


        df = pd.DataFrame(predictions)

        # Save to CSV file with a structured name
        filename = f"mutation_Pos{i}_{original_aa}_to_{new_aa}.csv"
        df.to_csv(filename, index=False)

    except Exception as e:
        print(f"Skipping mutation at position {i} ({original_aa} → {new_aa}): {e}")





Total mutations generated: 266


KeyboardInterrupt: 

In [ ]:
'''
Task3 - Run through the 280 csv files and extract the order of the kinases. Then using the test sequence as the reference.
Compute a metric for each pair using something similar to LCS (link for inspo attached - https://chatgpt.com/share/67c705a9-9c74-8001-a8da-eac57f06384f)  
'''

Extracted kinase orders from 286 CSV files.


In [ ]:
import os
import pandas as pd
import itertools
from difflib import SequenceMatcher
import os
import pandas as pd
import itertools
from difflib import SequenceMatcher

mut_predictions = "C:/Users/ricea/OneDrive/Documents/GitHub/Annabel-BSc-Project"
test_sequence = "PSVEPPLsQETFSDL"
kinase_orders = {}

for file in os.listdir(mut_predictions): #finds the csvs, reads the csvs
    if file.endswith(".csv"):
        file_path = os.path.join(mut_predictions, file)
        try:
            df = pd.read_csv(file_path)
            
            if "Score Rank" in df.columns: #finds the score rank column
                kinase_list = df["Score Rank"].tolist()
                kinase_orders[file] = kinase_list  # Store kinase order for each mutation
            else:
                print(f"Skipping {file}: No 'score rank' column found.") #proofreads
        
        except Exception as e:
            print(f"Error reading {file}: {e}")  #proofreads

print(f"Extracted kinase orders from {len(kinase_orders)} CSV files.") #checks its done


def longest_common_subsequence(seq1, seq2):
    """Computes the Longest Common Subsequence (LCS) length between two lists."""
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())


pairwise_lcs = {}
for (file1, order1), (file2, order2) in itertools.combinations(kinase_orders.items(), 2):
    lcs_length = longest_common_subsequence(order1, order2)
    pairwise_lcs[(file1, file2)] = lcs_length

#normalise lcs scores between 0 and 1
if pairwise_lcs:

    lcs_scores = list(pairwise_lcs.values())
    lcs_min = min(lcs_scores)
    lcs_max = max(lcs_scores)
    
    normalized_lcs = {pair: (score - lcs_min) / (lcs_max - lcs_min) for pair, score in pairwise_lcs.items()}

    # Step 3: Print top 5 most similar kinase orders with normalized LCS scores
    sorted_normalized_lcs = sorted(normalized_lcs.items(), key=lambda x: x[1], reverse=True)
    print("\nKinase orders (normalized LCS scores):")
    for (pair, normalized_score) in sorted_normalized_lcs:
        print(f"{pair[0]} vs {pair[1]}: Normalized LCS Score = {normalized_score:.4f}")
else:
    print("No LCS scores calculated. Ensure kinase_orders contains data.") #check kinase orders library actually has data in it


Extracted kinase orders from 286 CSV files.

Top 5 most similar kinase orders (normalized LCS scores):
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_C.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_D.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_E.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_F.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_G.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_H.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_I.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_K.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_L.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_M.csv: Normalized LCS Score = 1.0000
mutation_Pos0_P_to_A.csv vs mutation_Pos0_P_to_N.csv: Nor

In [ ]:
'''
Task 4: generate heatmaps of LCS ratio for each mutation on the y axis and position on the x axis. I like plotly heatmaps for this
https://plotly.com/python/heatmaps/. So output will be a heat map showing hotspots for the mutations that disrup the phosphorylation motif the most
'''

In [ ]:
#making the lcs data into a data frame for the heat map 
sorted_normalized_lcs.append([pair,normalized_score,])
df_lcs = pd.DataFrame(sorted_normalized_lcs, columns=['Mutation','LCS Ratio','Position'])
print(df_lcs)

# i need to work out how to change the positions out of the file name and into a matrix



                                                Mutation  LCS Ratio    
0      (mutation_Pos0_P_to_A.csv, mutation_Pos0_P_to_...        1.0 NaN
1      (mutation_Pos0_P_to_A.csv, mutation_Pos0_P_to_...        1.0 NaN
2      (mutation_Pos0_P_to_A.csv, mutation_Pos0_P_to_...        1.0 NaN
3      (mutation_Pos0_P_to_A.csv, mutation_Pos0_P_to_...        1.0 NaN
4      (mutation_Pos0_P_to_A.csv, mutation_Pos0_P_to_...        1.0 NaN
...                                                  ...        ...  ..
40770  (mutation_Pos7_s_to_Y.csv, mutation_Pos8_Q_to_...        0.0 NaN
40771  (mutation_Pos7_s_to_Y.csv, mutation_Pos8_Q_to_...        0.0 NaN
40772  (mutation_Pos7_s_to_Y.csv, mutation_Pos8_Q_to_...        0.0 NaN
40773  (mutation_Pos7_s_to_Y.csv, mutation_Pos8_Q_to_...        0.0 NaN
40774  (mutation_Pos7_s_to_Y.csv, mutation_Pos8_Q_to_...        0.0 NaN

[40775 rows x 3 columns]


PermissionError: [Errno 13] Permission denied: 'C:/Users/ricea/OneDrive/Documents/GitHub/Annabel-BSc-Project'

In [ ]:
import plotly as px

import numpy as np

import pandas as pd



C:/Users/ricea/OneDrive/Documents/GitHub/Annabel-BSc-Project
